In [1]:
import os
from os import path
import time

import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2

from sklearn.model_selection import ParameterGrid

import torchvision

import tensorflow as tf

from deepface import DeepFace

from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.tf2.attacks.carlini_wagner_l2 import carlini_wagner_l2
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent

from sklearn.metrics import pairwise_distances 

In [2]:
def init_deepface(data_dir_path, models, distance_metric, dataset='train'):
    for m in models:
        # print('finding:', os.path.join(data_dir_path, 'lfw-py', 'lfw_funneled',
        #                           'Aaron_Eckhart', 'Aaron_Eckhart_0001.jpg'))
        DeepFace.find(
            img_path=os.path.join(data_dir_path, 'lfw-py', 'lfw_funneled','Aaron_Eckhart', 'Aaron_Eckhart_0001.jpg'),
            db_path=os.path.join(data_dir_path, 'lfw-'+dataset),
            model_name=m,
            distance_metric=distance_metric,
            enforce_detection=False
        )
        
def get_model(model_name):
    # options:
    # VGG-Face, Google FaceNet, OpenFace, Facebook DeepFace, DeepID, ArcFace, Dlib.
    return DeepFace.build_model(model_name)

In [26]:
distance_metric = 'cosine'

model_name = 'VGG-Face'
eval_model_name = 'Facenet512'

model = get_model(model_name)
eval_model = get_model(eval_model_name)
input_shapes = (
    DeepFace.functions.find_input_shape(model), DeepFace.functions.find_input_shape(eval_model)
)


init_deepface('data', [model_name, eval_model_name], distance_metric, 'train')
init_deepface('data', [model_name, eval_model_name], distance_metric, 'test')

# load representations
train_rep = [
    pickle.load(open('data/lfw-train/representations_vgg_face.pkl', 'rb')),
    pickle.load(open('data/lfw-train/representations_facenet512.pkl','rb'))
]

test_rep = [
    pickle.load(open('data/lfw-test/representations_vgg_face.pkl','rb')),
    pickle.load(open('data/lfw-test/representations_facenet512.pkl','rb'))
]


There are  21878  representations found in  representations_vgg_face.pkl
find function lasts  28.32466959953308  seconds
There are  21878  representations found in  representations_facenet512.pkl
find function lasts  6.6727614402771  seconds
There are  2769  representations found in  representations_vgg_face.pkl
find function lasts  3.5300657749176025  seconds
There are  2769  representations found in  representations_facenet512.pkl
find function lasts  0.9545450210571289  seconds


In [7]:
# load param grid
params = pickle.load(open('attack_params.pkl', 'rb'))

In [27]:
attack_reps = {}

for a in params:
    for p in params[a]:
        param_str = a + '_' + '_'.join([str(p[k]) for k in sorted(list(p.keys()))])

        init_deepface('data', [model_name, eval_model_name], distance_metric, 'attack-3/'+param_str)

        attack_reps[param_str] = [
            pickle.load(open('data/lfw-attack-3/'+param_str+'/representations_vgg_face.pkl','rb')),
            pickle.load(open('data/lfw-attack-3/'+param_str+'/representations_facenet512.pkl','rb'))
        ]

There are  10  representations found in  representations_vgg_face.pkl
find function lasts  0.3489804267883301  seconds
There are  10  representations found in  representations_facenet512.pkl
find function lasts  0.11026930809020996  seconds
There are  10  representations found in  representations_vgg_face.pkl
find function lasts  0.13890886306762695  seconds
There are  10  representations found in  representations_facenet512.pkl
find function lasts  0.11602640151977539  seconds
There are  10  representations found in  representations_vgg_face.pkl
find function lasts  0.13002920150756836  seconds
There are  10  representations found in  representations_facenet512.pkl
find function lasts  0.10302305221557617  seconds
There are  10  representations found in  representations_vgg_face.pkl
find function lasts  0.1290292739868164  seconds
There are  10  representations found in  representations_facenet512.pkl
find function lasts  0.10302352905273438  seconds
There are  10  representations fou

In [9]:
def path_to_name(p):
    return os.path.split(os.path.split(p)[0])[-1]

In [28]:
x_train = [
    np.array([x[1] for x in train_rep[0]]),
    np.array([x[1] for x in train_rep[1]])
]
y_train = [
    [path_to_name(x[0]) for x in train_rep[0]],
    [path_to_name(x[0]) for x in train_rep[1]]
]
#----------------------------------------------
x_test = [
    np.array([x[1] for x in test_rep[0]]),
    np.array([x[1] for x in test_rep[1]]),
]
y_test =  [
    [path_to_name(x[0]) for x in test_rep[0]],
    [path_to_name(x[0]) for x in test_rep[1]],
]
#----------------------------------------------
x_attack = {}
y_attack = {}

for a in attack_reps:
    x_attack[a] = [
        np.array([x[1] for x in attack_reps[a][i]]) for i in [0, 1]
    ]
    y_attack[a] = [ 
        [path_to_name(x[0]) for x in attack_reps[a][i]] for i in [0,1]
    ]

# get memory back
# del train_rep, test_rep, attack_reps

In [29]:
# for each attack image, compute benchmark and attack performance on both model and eval

# benchmark: test vs train 
test_dist = [
    pairwise_distances(x_test[i], x_train[i]) for i in [0,1]
]
# accuracy: attack vs train
attack_dist = {
    a : [pairwise_distances(x_attack[a][i], x_train[i]) for i in [0,1]] for a in x_attack
}

In [30]:
# compute the performance using the pairwise distances
def classify(ys, ypreds, dist):
    correct = []
    for i, row in enumerate(dist):
        ypred = ypreds[row.argmin()]
        
        correct.append(int(ypred == ys[i]))
    return correct

In [31]:
# compute the best performing attack
test_correct = [classify(y_test[i], y_train[i], test_dist[i]) for i in [0,1]]

In [38]:
attack_correct = {
    a : [classify(y_attack[a][i], y_train[i], attack_dist[a][i]) for i in [0,1]] for a in x_attack
}

In [33]:
print([(sum(test_correct[i]), sum(test_correct[i])/len(test_correct[i]))for i in [0,1]])

for a in attack_correct:
    print([(sum(attack_correct[a][i]), sum(attack_correct[a][i])/len(attack_correct[a][i])) for i in [0,1]])

[(2765, 0.9985554351751534), (2766, 0.9989165763813651)]
[(9, 0.9), (10, 1.0)]
[(10, 1.0), (10, 1.0)]
[(8, 0.8), (8, 0.8)]
[(9, 0.9), (9, 0.9)]
[(9, 0.9), (9, 0.9)]
[(9, 0.9), (9, 0.9)]
[(10, 1.0), (10, 1.0)]
[(8, 0.8), (10, 1.0)]
[(9, 0.9), (10, 1.0)]
[(9, 0.9), (9, 0.9)]
[(7, 0.7), (6, 0.6)]
[(8, 0.8), (9, 0.9)]
[(9, 0.9), (9, 0.9)]
[(10, 1.0), (10, 1.0)]
[(7, 0.7), (7, 0.7)]
[(9, 0.9), (9, 0.9)]
[(9, 0.9), (9, 0.9)]
[(9, 0.9), (10, 1.0)]
[(10, 1.0), (10, 1.0)]
[(8, 0.8), (9, 0.9)]
[(8, 0.8), (8, 0.8)]
[(10, 1.0), (10, 1.0)]
[(7, 0.7), (6, 0.6)]
[(10, 1.0), (10, 1.0)]
[(10, 1.0), (10, 1.0)]
[(5, 0.5), (8, 0.8)]
[(8, 0.8), (8, 0.8)]
[(6, 0.6), (9, 0.9)]
[(9, 0.9), (9, 0.9)]
[(0, 0.0), (0, 0.0)]
[(10, 1.0), (10, 1.0)]
[(0, 0.0), (0, 0.0)]


In [47]:
best = min([[sum(c[0]), sum(c[1])] for c in attack_correct.values()])
print(best)
for a,c in attack_correct.items():
    if sum(c[0]) == best[0] or sum(c[1]) == best[1]:
        print(a, sum(c[0]), sum(c[1]))

[0, 0]
pgd_0.1_0.01_100_inf 0 0
pgd_0.1_0.1_100_inf 0 0


In [50]:
attack_correct['pgd_0.001_0.1_100_2']

[[1, 0, 0, 0, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 1, 1, 1, 0, 1, 1]]